# Использование TfidfVectorizer для получения относительных частот имен отношений в тексте

In [100]:
# сначала откроем все списки, которые понадобятся для создания таблицы и для проверки результата
# не знаю, что за библиотека ast, но только с ее помощью получилось открыть записанный в файл список как список, а не как строку
with open("texts_in_corpus_links.txt", "r", encoding='utf-8') as file:
    text = file.read()
import ast
texts_in_corpus_links = ast.literal_eval(text)

with open("corpus_files_names.txt", "r", encoding='utf-8') as file:
    text = file.read()
corpus_files_names = ast.literal_eval(text)

with open("prince_links.txt", "r", encoding='utf-8') as file:
    text = file.read()
prince_links = ast.literal_eval(text)

with open("corpus_links.txt", "r", encoding='utf-8') as file:
    text = file.read()
corpus_links = ast.literal_eval(text)

In [101]:
# проверяю, что правильно открылся спислок перечней лемм
# он специально сохранен как список строк, потому что что sklearn работает с текстами, а не со списками
# каждая строка содержит все леммы одного файла
len(texts_in_corpus_links)

1304

In [102]:
# еще раз проверяю, что правильно открылся список перечней связей
print(texts_in_corpus_links[0][0:10])

обст предл


In [103]:
# зовем sklearn, документация:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# делала то в Colab, то в Jupyter, в одном работало sklearn, в другом -U scikit-learn
!pip3 install sklearn
!pip3 install -U scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip


In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

* по умолчанию в TfidfVectorizer стоит такой параметр: token_pattern='(?u)\b\w\w+\b'
 (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
,
* в отличие от лемм, тут однобуквенных не будет штук
* но дефисы нужны (ср. связь соч-союзн)
* тот token_pattern, который был для лемм здесь, сначала оставила, потом изменила
* не прошло отношение 1-несобст-компл - расширила еще token_pattern
* а вот ограничение на df пока не делаю

In [105]:
# задаем наши параметры - схема токена
# и отбрасываемые слишком частые слова
tfidf_vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+-?\w*-?\w*\b")

In [106]:
# применяем эту штуку с нашими параметрами к списку перечней лемм
tfidf = tfidf_vectorizer.fit_transform(texts_in_corpus_links)

In [107]:
tfidf.shape

(1304, 70)

In [108]:
links = tfidf_vectorizer.get_feature_names_out()
len(links)

70

In [109]:
links

array(['1-компл', '1-несобст-компл', '2-компл', '2-несобст-компл',
       '3-компл', '3-несобст-компл', '4-компл', '4-несобст-компл',
       '5-компл', 'root', 'агент', 'аддит', 'адр-присв', 'аналит',
       'аппоз', 'аппрокс-колич', 'аппрокс-порядк', 'атриб', 'вводн',
       'вспом', 'дат-сент', 'дат-субъект', 'дистанц', 'длительн',
       'изъясн', 'инф-союзн', 'квазиагент', 'колич-вспом', 'колич-копред',
       'колич-огран', 'количест', 'компл-аппоз', 'композ', 'кратн',
       'кратно-длительн', 'неакт-компл', 'несобст-агент', 'ном-аппоз',
       'нум-аппоз', 'об-аппоз', 'об-копр', 'об-обст', 'обст', 'обст-тавт',
       'огранич', 'оп-аппоз', 'оп-опред', 'опред', 'пасс-анал',
       'подч-союзн', 'предик', 'предл', 'примыкат', 'присвяз', 'пролепт',
       'разъяснит', 'распред', 'релят', 'сент-предик', 'сент-соч',
       'соотнос', 'соч-союзн', 'сочин', 'сравн-союзн', 'сравнит',
       'суб-копр', 'суб-обст', 'уточн', 'эксплет', 'электив'],
      dtype=object)

In [110]:
!pip3 install pandas


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip


In [111]:
import pandas as pd

In [112]:
# формируем таблицу со словами и именами файлов
data = tfidf.todense().tolist() 
SynTagRusTable_links = pd.DataFrame(data, columns = links, index = corpus_files_names)

In [113]:
#строка, соответствующая нашему тексту
print(len(SynTagRusTable_links.loc['Malenkii_prints']))

70


In [114]:
# сохраним в переменную
prints_tf_list = SynTagRusTable_links.loc['Malenkii_prints']

In [115]:
# проверим, что внутри
prints_tf_list

1-компл            0.383906
1-несобст-компл    0.002352
2-компл            0.092389
2-несобст-компл    0.000000
3-компл            0.020907
                     ...   
суб-копр           0.014203
суб-обст           0.000000
уточн              0.000000
эксплет            0.010594
электив            0.002285
Name: Malenkii_prints, Length: 70, dtype: float64

In [116]:
# проверяю, что "1-несобст-компл" входит после правки
prints_tf_list["1-несобст-компл"]

0.002352248030242429

In [118]:
# еще один способ извлечь максимальное значение
prints_tf_list.max()

0.5192600124602422

In [119]:
# можно сортировать
#Series.sort_values ​​( ось = 0 , по возрастанию = True , inplace = False , kind = 'quicksort' , na_position = 'last' , ignore_index = False , key = None ) 
prints_tf_list.sort_values(ascending=False)

предик             0.519260
root               0.422950
1-компл            0.383906
огранич            0.319222
опред              0.274157
                     ...   
несобст-агент      0.000000
дистанц            0.000000
изъясн             0.000000
колич-копред       0.000000
кратно-длительн    0.000000
Name: Malenkii_prints, Length: 70, dtype: float64

In [120]:
# вижу, что нули есть
# вычеркиваю
unempty_values_links = prints_tf_list.loc[lambda x: x != 0.0]

In [121]:
# проверяем, что количество уменьшилось
len(unempty_values_links)

52

In [122]:
# проверяем, что внутри
unempty_values_links

1-компл            0.383906
1-несобст-компл    0.002352
2-компл            0.092389
3-компл            0.020907
root               0.422950
агент              0.000419
аддит              0.001355
адр-присв          0.002758
аналит             0.021187
аппоз              0.004885
аппрокс-колич      0.000834
атриб              0.032575
вводн              0.101388
вспом              0.018940
дат-субъект        0.030887
длительн           0.009512
инф-союзн          0.013104
квазиагент         0.035841
колич-вспом        0.036770
колич-огран        0.003994
количест           0.031471
компл-аппоз        0.001904
кратн              0.002502
неакт-компл        0.006539
нум-аппоз          0.008537
об-аппоз           0.003329
об-копр            0.002654
обст               0.273824
обст-тавт          0.001107
огранич            0.319222
оп-опред           0.006698
опред              0.274157
пасс-анал          0.001933
подч-союзн         0.089595
предик             0.519260
предл              0

In [123]:
# посмотрим в порядке убывания, заодно проверим, что нули действительно не вошли
unempty_values_links.sort_values(ascending=False)

предик             0.519260
root               0.422950
1-компл            0.383906
огранич            0.319222
опред              0.274157
обст               0.273824
предл              0.239012
соч-союзн          0.195920
сочин              0.101625
вводн              0.101388
2-компл            0.092389
подч-союзн         0.089595
сент-соч           0.074823
присвяз            0.058792
колич-вспом        0.036770
квазиагент         0.035841
атриб              0.032575
количест           0.031471
дат-субъект        0.030887
разъяснит          0.029215
релят              0.024801
сравнит            0.023220
сравн-союзн        0.021630
аналит             0.021187
3-компл            0.020907
вспом              0.018940
суб-копр           0.014203
инф-союзн          0.013104
эксплет            0.010594
длительн           0.009512
нум-аппоз          0.008537
пролепт            0.007542
соотнос            0.007310
оп-опред           0.006698
неакт-компл        0.006539
аппоз              0

*Теперь проверяем разницу между исходным списком и тем, которые получился в результате процедуры подсчета  tfidf*

In [124]:
# превращаем упорядоченный Series в словарь
prince_links_dict = unempty_values_links.sort_values(ascending=False).to_dict()

In [125]:
# получилось? да
type(prince_links_dict)

dict

In [128]:
# из словаря можно сделать список ключей - имена отношений выживших после процедуры
list(prince_links_dict.keys())

['предик',
 'root',
 '1-компл',
 'огранич',
 'опред',
 'обст',
 'предл',
 'соч-союзн',
 'сочин',
 'вводн',
 '2-компл',
 'подч-союзн',
 'сент-соч',
 'присвяз',
 'колич-вспом',
 'квазиагент',
 'атриб',
 'количест',
 'дат-субъект',
 'разъяснит',
 'релят',
 'сравнит',
 'сравн-союзн',
 'аналит',
 '3-компл',
 'вспом',
 'суб-копр',
 'инф-союзн',
 'эксплет',
 'длительн',
 'нум-аппоз',
 'пролепт',
 'соотнос',
 'оп-опред',
 'неакт-компл',
 'аппоз',
 'колич-огран',
 'об-аппоз',
 'распред',
 'адр-присв',
 'об-копр',
 'кратн',
 '1-несобст-компл',
 'электив',
 'сент-предик',
 'пасс-анал',
 'компл-аппоз',
 'примыкат',
 'аддит',
 'обст-тавт',
 'аппрокс-колич',
 'агент']

In [129]:
print(list(prince_links_dict.keys())[0:49])

['предик', 'root', '1-компл', 'огранич', 'опред', 'обст', 'предл', 'соч-союзн', 'сочин', 'вводн', '2-компл', 'подч-союзн', 'сент-соч', 'присвяз', 'колич-вспом', 'квазиагент', 'атриб', 'количест', 'дат-субъект', 'разъяснит', 'релят', 'сравнит', 'сравн-союзн', 'аналит', '3-компл', 'вспом', 'суб-копр', 'инф-союзн', 'эксплет', 'длительн', 'нум-аппоз', 'пролепт', 'соотнос', 'оп-опред', 'неакт-компл', 'аппоз', 'колич-огран', 'об-аппоз', 'распред', 'адр-присв', 'об-копр', 'кратн', '1-несобст-компл', 'электив', 'сент-предик', 'пасс-анал', 'компл-аппоз', 'примыкат', 'аддит']


In [130]:
list(prince_links_dict.items())[0:20]

[('предик', 0.5192600124602422),
 ('root', 0.4229500310478148),
 ('1-компл', 0.3839062196717479),
 ('огранич', 0.3192221788450057),
 ('опред', 0.2741572264846),
 ('обст', 0.27382354933634195),
 ('предл', 0.23901153133619296),
 ('соч-союзн', 0.1959198254033002),
 ('сочин', 0.10162479897436331),
 ('вводн', 0.10138763551572115),
 ('2-компл', 0.09238873441003558),
 ('подч-союзн', 0.08959507556841274),
 ('сент-соч', 0.07482288093548276),
 ('присвяз', 0.05879169346170164),
 ('колич-вспом', 0.036769972419030635),
 ('квазиагент', 0.035840577996213145),
 ('атриб', 0.03257463828016295),
 ('количест', 0.03147146615053359),
 ('дат-субъект', 0.03088671802019275),
 ('разъяснит', 0.029214598380231365)]

Скромные выводы по этому списку:
* первые три строчки соответствуют тому, что в этом тексте относительно короткие предложения
* остальное невнятно

In [131]:
# связи до были капитализированы, если хотим сравнивать, надо сделать строчными
def all_lower(my_list):
    return [x.lower() for x in my_list]

all_lower(prince_links)[0:10]

['root',
 'вспом',
 'аппоз',
 'опред',
 'root',
 'root',
 'аппоз',
 'сочин',
 'root',
 'аппоз']

In [132]:
# посмотрим, сколько вычеркнули
difference = list(set(all_lower(prince_links)) - set(list(prince_links_dict.keys())))
len(difference)

0

In [133]:
# не попало название с двумя дефисами 1-несобст-компл, правим token_pattern
# после исправления разницы нет
print(difference)

[]


*повторяем для max = 0.7,*
*ну в самом деле, мало ли что*

In [134]:

tfidf_vectorizer_07 = TfidfVectorizer(token_pattern=r"(?u)\b\w+-?\w*-?\w*\b", max_df = 0.7)

In [135]:
tfidf_07 = tfidf_vectorizer_07.fit_transform(texts_in_corpus_links)

In [136]:
# проверяем параметры - на 20 связей слов меньше
tfidf_07.shape

(1304, 50)

In [137]:
words_07 = tfidf_vectorizer_07.get_feature_names_out()
len(words_07)

50

In [138]:
# формируем таблицу со словами и именами файлов
data = tfidf_07.todense().tolist() 
SynTagRusTable_07 = pd.DataFrame(data, columns = words_07, index = corpus_files_names)

In [139]:
# строка для нашего текста
prints_tf_list_07 = SynTagRusTable_07.loc['Malenkii_prints']

In [140]:
# вычеркивавем нулевые значения
unempty_values_links_07 = prints_tf_list_07.loc[lambda x: x != 0.0]

In [141]:
# превращаем упорядоченный Series в словарь
prince_links_dict_07 = unempty_values_links_07.sort_values(ascending=False).to_dict()

In [142]:
len(prince_links_dict_07.keys())

32

In [143]:
print(list(prince_links_dict_07.keys()))

['присвяз', 'колич-вспом', 'дат-субъект', 'разъяснит', 'сравнит', 'аналит', 'вспом', 'суб-копр', 'инф-союзн', 'эксплет', 'длительн', 'нум-аппоз', 'пролепт', 'соотнос', 'оп-опред', 'неакт-компл', 'колич-огран', 'об-аппоз', 'распред', 'адр-присв', 'об-копр', 'кратн', '1-несобст-компл', 'электив', 'сент-предик', 'пасс-анал', 'компл-аппоз', 'примыкат', 'аддит', 'обст-тавт', 'аппрокс-колич', 'агент']


In [144]:
# посмотрим, есть ли разница, для этого нужно сделать список множеством - убрать повторяющиеся
difference_07 = list(set(all_lower(prince_links)) - set(list(prince_links_dict_07.keys())))
len(difference_07)

20

In [145]:
print(difference_07)

['сравн-союзн', 'количест', 'огранич', 'релят', 'опред', 'сент-соч', 'подч-союзн', 'аппоз', '2-компл', '1-компл', 'root', 'обст', 'сочин', '3-компл', 'соч-союзн', 'вводн', 'квазиагент', 'предик', 'предл', 'атриб']


эти результаты забавны, но тоже не очень внятны
* в результате мы получили список связей, которые присутствуют почти во всех текстах. Но ради этого, кажется, не стоило такой огород городить